In [64]:
#Phase 1: Data Exploration & Preprocessing

import pandas as pd
import os

# Function to load OULAD data from CSV files
# Using raw strings to handle backslashes in Windows file paths
# This function assumes the CSV files are located in the specified directory
# and that the directory structure matches the expected format.

def load_oulad_data(data_path):
    """Load all OULAD CSV files into pandas DataFrames"""

    # Core tables - using raw strings
    courses = pd.read_csv(rf"{data_path}\courses.csv")
    assessments = pd.read_csv(rf"{data_path}\assessments.csv")
    vle = pd.read_csv(rf"{data_path}\vle.csv")

    # Student data
    student_info = pd.read_csv(rf"{data_path}\studentInfo.csv")
    student_registration = pd.read_csv(rf"{data_path}\studentRegistration.csv")
    student_assessment = pd.read_csv(rf"{data_path}\studentAssessment.csv")
    student_vle = pd.read_csv(rf"{data_path}\studentVle.csv")

    return {
        'courses': courses,
        'assessments': assessments,
        'vle': vle,
        'student_info': student_info,
        'student_registration': student_registration,
        'student_assessment': student_assessment,
        'student_vle': student_vle
    }

# Load the data - use raw string for path
oulad_data = load_oulad_data(r"C:\Users\Ritam\Projects\XAIDashboard\dataset")

In [66]:
#data exploration
# Function to explore the main student information table
def explore_student_data(oulad_data):
    """Explore the main student information table"""

    student_info = oulad_data['student_info']

    print("=== STUDENT INFO EXPLORATION ===")
    print(f"Shape: {student_info.shape}")
    print(f"Columns: {list(student_info.columns)}")
    print("\n--- Sample Data ---")
    print(student_info.head(3))

    print("\n--- Target Variable Distribution ---")
    print(student_info['final_result'].value_counts())
    print(student_info['final_result'].value_counts(normalize=True).round(3))

    print("\n--- Demographic Breakdown ---")
    print("Gender:", student_info['gender'].value_counts().to_dict())
    print("Age bands:", student_info['age_band'].value_counts().to_dict())
    print("Disability:", student_info['disability'].value_counts().to_dict())

# Run exploration
explore_student_data(oulad_data)

=== STUDENT INFO EXPLORATION ===
Shape: (32593, 12)
Columns: ['code_module', 'code_presentation', 'id_student', 'gender', 'region', 'highest_education', 'imd_band', 'age_band', 'num_of_prev_attempts', 'studied_credits', 'disability', 'final_result']

--- Sample Data ---
  code_module code_presentation  id_student gender                region  \
0         AAA             2013J       11391      M   East Anglian Region   
1         AAA             2013J       28400      F              Scotland   
2         AAA             2013J       30268      F  North Western Region   

       highest_education imd_band age_band  num_of_prev_attempts  \
0       HE Qualification  90-100%     55<=                     0   
1       HE Qualification   20-30%    35-55                     0   
2  A Level or Equivalent   30-40%    35-55                     0   

   studied_credits disability final_result  
0              240          N         Pass  
1               60          N         Pass  
2               

In [67]:
#Check VLE Engagement Patterns
def explore_vle_data(oulad_data):
    """Explore VLE interaction patterns"""

    student_vle = oulad_data['student_vle']

    print("=== VLE INTERACTION EXPLORATION ===")
    print(f"Total interactions: {len(student_vle):,}")
    print(f"Unique students with VLE data: {student_vle['id_student'].nunique():,}")

    # Basic statistics
    print(f"Clicks per interaction - Mean: {student_vle['sum_click'].mean():.1f}, Max: {student_vle['sum_click'].max()}")

    # Activity patterns
    activity_types = oulad_data['vle']['activity_type'].value_counts().head(10)
    print("\n--- Top Activity Types ---")
    print(activity_types)

# Run VLE exploration
explore_vle_data(oulad_data)



=== VLE INTERACTION EXPLORATION ===
Total interactions: 10,655,280
Unique students with VLE data: 26,074
Clicks per interaction - Mean: 3.7, Max: 6977

--- Top Activity Types ---
activity_type
resource         2660
subpage          1055
oucontent         996
url               886
forumng           194
quiz              127
page              102
oucollaborate      82
questionnaire      61
ouwiki             49
Name: count, dtype: int64


In [68]:
#Assess Data Quality
def assess_data_quality(oulad_data):
    """Quick data quality assessment"""

    print("=== DATA QUALITY ASSESSMENT ===")

    for table_name, df in oulad_data.items():
        missing_pct = (df.isnull().sum().sum() / (len(df) * len(df.columns))) * 100
        print(f"{table_name}: {missing_pct:.1f}% missing values")

        if missing_pct > 0:
            missing_cols = df.isnull().sum()
            critical_missing = missing_cols[missing_cols > len(df) * 0.1]  # >10% missing
            if len(critical_missing) > 0:
                print(f"  ⚠️  High missingness in: {critical_missing.to_dict()}")

# Run quality assessment
assess_data_quality(oulad_data)


=== DATA QUALITY ASSESSMENT ===
courses: 0.0% missing values
assessments: 0.9% missing values
vle: 27.5% missing values
  ⚠️  High missingness in: {'week_from': 5243, 'week_to': 5243}
student_info: 0.3% missing values
student_registration: 13.8% missing values
  ⚠️  High missingness in: {'date_unregistration': 22521}
student_assessment: 0.0% missing values
student_vle: 0.0% missing values


In [69]:
# VLE Table: week_from and week_to (27.5% missing)
def handle_vle_missing(vle_data):
    """Handle missing week data in VLE table"""

    # Strategy 1: Create 'continuous' category for missing weeks
    vle_data['week_from'] = vle_data['week_from'].fillna(-1)  # -1 = continuous
    vle_data['week_to'] = vle_data['week_to'].fillna(-1)

    # Strategy 2: Create boolean flag for continuous activities
    vle_data['is_continuous_activity'] = vle_data['week_from'].isna()

    # Strategy 3: Fill with course-wide availability (0 to max_week)
    max_week = vle_data['week_to'].max()
    vle_data['week_from_filled'] = vle_data['week_from'].fillna(0)
    vle_data['week_to_filled'] = vle_data['week_to'].fillna(max_week)

    return vle_data

#Student Registration: date_unregistration (13.8% missing)
def handle_registration_missing(registration_data):
    """Handle missing unregistration dates"""

    # Strategy 1: Create binary completion indicator
    registration_data['completed_course'] = registration_data['date_unregistration'].isna()

    # Strategy 2: Fill with end-of-course date for completed students
    # (You can get course end dates from course table)
    registration_data['unregistration_status'] = registration_data['date_unregistration'].apply(
        lambda x: 'completed' if pd.isna(x) else 'withdrew'
    )

    # Strategy 3: Calculate retention days (for non-missing values)
    registration_data['retention_days'] = (
        registration_data['date_unregistration'] - registration_data['date_registration']
    ).dt.days

    return registration_data


In [70]:
#Handle all missing values comprehensively
def comprehensive_missing_value_handling(oulad_data):
    """Complete strategy for handling all missing values"""

    print("=== HANDLING MISSING VALUES ===")

    # 1. VLE table - handle week missingness
    print("Processing VLE missing values...")
    vle = oulad_data['vle'].copy()

    # Create flags for continuous activities
    vle['is_continuous'] = vle['week_from'].isna()

    # Fill with meaningful values
    vle['week_from'] = vle['week_from'].fillna(0)  # Start of course
    vle['week_to'] = vle['week_to'].fillna(vle['week_to'].max())  # End of course

    oulad_data['vle'] = vle
    print(f"✅ VLE: Added continuous activity flags")

    # 2. Student registration - handle unregistration missingness
    print("Processing registration missing values...")
    registration = oulad_data['student_registration'].copy()

    # Create completion indicators
    registration['completed_course'] = registration['date_unregistration'].isna()
    registration['withdrawal_status'] = registration['date_unregistration'].apply(
        lambda x: 'completed' if pd.isna(x) else 'withdrew_early'
    )

    oulad_data['student_registration'] = registration
    print(f"✅ Registration: Added completion indicators")

    # 3. Handle minor missing values in other tables
    print("Processing minor missing values...")

    # Student info - forward fill demographic data (assumed stable)
    student_info = oulad_data['student_info'].copy()
    demographic_cols = ['gender', 'region', 'highest_education', 'disability']
    for col in demographic_cols:
        if col in student_info.columns and student_info[col].isna().any():
            # Fill with mode (most common value)
            mode_value = student_info[col].mode()[0]
            student_info[col] = student_info[col].fillna(mode_value)
            print(f"   Filled {col} with mode: {mode_value}")

    oulad_data['student_info'] = student_info

    # 4. Verify no critical missing values remain
    print("\n=== POST-PROCESSING VERIFICATION ===")
    for table_name, df in oulad_data.items():
        critical_missing = df.isnull().sum().sum()
        if critical_missing > 0:
            print(f"⚠️  {table_name}: {critical_missing} missing values remain")
            missing_cols = df.isnull().sum()
            print(f"   Columns: {missing_cols[missing_cols > 0].to_dict()}")
        else:
            print(f"✅ {table_name}: No missing values")

    return oulad_data

# Apply the comprehensive handling
oulad_data_clean = comprehensive_missing_value_handling(oulad_data)


=== HANDLING MISSING VALUES ===
Processing VLE missing values...
✅ VLE: Added continuous activity flags
Processing registration missing values...
✅ Registration: Added completion indicators
Processing minor missing values...

=== POST-PROCESSING VERIFICATION ===
✅ courses: No missing values
⚠️  assessments: 11 missing values remain
   Columns: {'date': 11}
✅ vle: No missing values
⚠️  student_info: 1111 missing values remain
   Columns: {'imd_band': 1111}
⚠️  student_registration: 22566 missing values remain
   Columns: {'date_registration': 45, 'date_unregistration': 22521}
⚠️  student_assessment: 173 missing values remain
   Columns: {'score': 173}
✅ student_vle: No missing values


In [71]:
#4. Feature Engineering from Missing Value Patterns
def create_missing_value_features(oulad_data_clean):
    """Create predictive features from missing value patterns"""

    # From VLE data
    vle_features = oulad_data_clean['vle'].groupby(['code_module', 'code_presentation']).agg({
        'is_continuous': 'sum',  # Count of continuous activities per course
    }).reset_index()
    vle_features.rename(columns={'is_continuous': 'continuous_activities_count'}, inplace=True)

    # From registration data
    registration_features = oulad_data_clean['student_registration'][
        ['code_module', 'code_presentation', 'id_student', 'completed_course', 'withdrawal_status']
    ]

    print("✅ Created predictive features from missing value patterns")

    return vle_features, registration_features

def verify_missing_handling(original_data, clean_data):
    """Verify missing value handling worked correctly"""

    print("=== MISSING VALUE HANDLING VERIFICATION ===")

    for table_name in original_data.keys():
        original_missing = original_data[table_name].isnull().sum().sum()
        clean_missing = clean_data[table_name].isnull().sum().sum()

        print(f"{table_name}:")
        print(f"  Before: {original_missing} missing values")
        print(f"  After:  {clean_missing} missing values")
        print(f"  Reduction: {original_missing - clean_missing}")

        # Check if any new features were created
        original_cols = set(original_data[table_name].columns)
        clean_cols = set(clean_data[table_name].columns)
        new_features = clean_cols - original_cols
        if new_features:
            print(f"  ✅ New features: {new_features}")
        print()

# Run verification
verify_missing_handling(oulad_data, oulad_data_clean)



=== MISSING VALUE HANDLING VERIFICATION ===
courses:
  Before: 0 missing values
  After:  0 missing values
  Reduction: 0

assessments:
  Before: 11 missing values
  After:  11 missing values
  Reduction: 0

vle:
  Before: 0 missing values
  After:  0 missing values
  Reduction: 0

student_info:
  Before: 1111 missing values
  After:  1111 missing values
  Reduction: 0

student_registration:
  Before: 22566 missing values
  After:  22566 missing values
  Reduction: 0

student_assessment:
  Before: 173 missing values
  After:  173 missing values
  Reduction: 0

student_vle:
  Before: 0 missing values
  After:  0 missing values
  Reduction: 0



In [72]:
#Phase 2: Predictive Modeling
#Phase 2A: Data Integration & Feature Engineering
import numpy as np
#Step 1:Combined Dataset
def create_combined_dataset(oulad_data_clean):
    """Combine all OULAD tables into a single analysis-ready dataset"""

    print("=== CREATING COMBINED DATASET ===")

    # Start with student_info as base table
    combined = oulad_data_clean['student_info'].copy()

    # Add registration data with our new completion features
    registration = oulad_data_clean['student_registration'][
        ['code_module', 'code_presentation', 'id_student', 'completed_course', 'withdrawal_status']
    ]
    combined = combined.merge(registration, on=['code_module', 'code_presentation', 'id_student'], how='inner')

    # Aggregate VLE engagement features
    vle_features = create_vle_features_safe(oulad_data_clean['student_vle'])
    combined = combined.merge(vle_features, on='id_student', how='left')

    # Aggregate assessment performance features
    assessment_features = create_assessment_features(oulad_data_clean['student_assessment'])
    combined = combined.merge(assessment_features, on='id_student', how='left')

    return combined

# Implement this first
combined_dataset = create_combined_dataset(oulad_data_clean)


=== CREATING COMBINED DATASET ===
🔄 Creating VLE features with safe calculations...
✅ VLE features created for 26074 students


In [79]:
def fix_infinite_and_extreme_values(df, features):
    """Clean infinite and extreme values from dataset"""

    print("🧹 CLEANING INFINITE AND EXTREME VALUES")

    df_clean = df.copy()
    issues_found = {}

    for col in features:
        if col in df_clean.columns:
            # Check for issues
            infinite_count = np.isinf(df_clean[col]).sum()
            extreme_count = (df_clean[col].abs() > 1e10).sum()

            if infinite_count > 0 or extreme_count > 0:
                issues_found[col] = {'infinite': infinite_count, 'extreme': extreme_count}

                # Replace infinite values with NaN
                df_clean[col] = df_clean[col].replace([np.inf, -np.inf], np.nan)

                # Cap extreme values at 99.9th percentile
                if extreme_count > 0:
                    upper_cap = df_clean[col].quantile(0.999)
                    lower_cap = df_clean[col].quantile(0.001)
                    df_clean[col] = df_clean[col].clip(lower=lower_cap, upper=upper_cap)

                print(f"   ✅ Fixed {col}: {infinite_count} infinite, {extreme_count} extreme values")

    # Fill remaining missing values with median
    for col in features:
        if col in df_clean.columns and df_clean[col].isnull().sum() > 0:
            median_val = df_clean[col].median()
            df_clean[col] = df_clean[col].fillna(median_val)
            print(f"   ✅ Filled {col} missing values with median: {median_val:.2f}")

    print(f"✅ Data cleaning complete. Issues found in: {list(issues_found.keys())}")
    return df_clean, issues_found


In [80]:
#Step 2: VLE Behavioral Feature Engineering
def create_vle_features_safe(student_vle):
    """Create VLE engagement features with safe division"""

    print("🔄 Creating VLE features with safe calculations...")

    vle_features = student_vle.groupby('id_student').agg({
        'sum_click': ['sum', 'mean', 'std', 'count'],
        'date': ['min', 'max', 'nunique']
    }).reset_index()

    # Flatten column names
    vle_features.columns = ['id_student', 'total_clicks', 'avg_clicks_per_session',
                           'click_variability', 'total_sessions', 'first_access_day',
                           'last_access_day', 'active_days']

    # Create derived features with safe division
    vle_features['engagement_duration'] = (
        vle_features['last_access_day'] - vle_features['first_access_day']
    )

    # Safe division for daily engagement rate
    vle_features['daily_engagement_rate'] = np.where(
        vle_features['engagement_duration'] > 0,
        vle_features['active_days'] / vle_features['engagement_duration'],
        0  # Default to 0 if duration is 0
    )

    # Cap engagement rate at reasonable maximum (e.g., 1.0)
    vle_features['daily_engagement_rate'] = vle_features['daily_engagement_rate'].clip(0, 1.0)

    # Fill NaN values from std calculation
    vle_features['click_variability'] = vle_features['click_variability'].fillna(0)

    print(f"✅ VLE features created for {len(vle_features)} students")
    return vle_features



In [81]:
#Step 3: Assessment Performance Features
def create_assessment_features(student_assessment):
    """Create assessment performance and timing features"""

    assessment_features = student_assessment.groupby('id_student').agg({
        'score': ['mean', 'std', 'count'],
        'date_submitted': ['min', 'max'],
        'is_banked': 'sum'
    }).reset_index()

    # Flatten columns
    assessment_features.columns = ['id_student', 'avg_assessment_score', 'score_consistency',
                                  'total_assessments', 'first_submission', 'last_submission',
                                  'banked_assessments']

    return assessment_features


In [82]:
#Phase 2B: Model Training Pipeline (This Week)
#Step 4: Feature Selection & Preparation

def prepare_modeling_data(combined_dataset):
    """Prepare data for machine learning"""

    # Define feature categories
    demographic_features = ['gender', 'age_band', 'highest_education', 'disability', 'region']
    behavioral_features = ['total_clicks', 'avg_clicks_per_session', 'active_days', 'daily_engagement_rate']
    academic_features = ['avg_assessment_score', 'total_assessments', 'studied_credits']
    engagement_features = ['completed_course', 'total_sessions', 'engagement_duration']

    # Create feature sets for comparison
    all_features = demographic_features + behavioral_features + academic_features + engagement_features
    behavior_only = demographic_features + behavioral_features + engagement_features  # No grades

    # Handle categorical variables
    from sklearn.preprocessing import LabelEncoder, StandardScaler

    # Encode categorical variables
    le_dict = {}
    processed_data = combined_dataset.copy()

    for col in demographic_features:
        if processed_data[col].dtype == 'object':
            le = LabelEncoder()
            processed_data[col] = le.fit_transform(processed_data[col].astype(str))
            le_dict[col] = le

    # Handle target variable
    target_encoder = LabelEncoder()
    processed_data['target'] = target_encoder.fit_transform(processed_data['final_result'])

    return processed_data, all_features, behavior_only, target_encoder, le_dict

#Step 5: Train-Test Split & Baseline Models
def train_baseline_models_safe(processed_data, feature_sets, target_col='target'):
    """Train models with comprehensive data validation"""

    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.linear_model import LogisticRegression
    from xgboost import XGBClassifier
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import classification_report, accuracy_score, f1_score
    import numpy as np
    import warnings
    warnings.filterwarnings('ignore')

    print("🚀 STARTING SAFE BASELINE MODEL TRAINING")
    print(f"Dataset shape: {processed_data.shape}")
    print(f"Target distribution:\n{processed_data[target_col].value_counts()}")

    results = {}

    for feature_set_name, features in feature_sets.items():
        print(f"\n{'='*60}")
        print(f"🎯 TRAINING ON {feature_set_name.upper()} FEATURE SET")
        print(f"{'='*60}")

        # Prepare data with validation
        X = processed_data[features].copy()
        y = processed_data[target_col].copy()

        print(f"✅ Initial data: X={X.shape}, y={y.shape}")

        # CRITICAL: Clean infinite and extreme values
        X_clean, issues = fix_infinite_and_extreme_values(X, features)

        # Final validation
        infinite_check = np.isinf(X_clean.select_dtypes(include=[np.number])).any().any()
        missing_check = X_clean.isnull().any().any()

        if infinite_check:
            print("❌ Still have infinite values after cleaning!")
            return None

        if missing_check:
            print("⚠️  Still have missing values - final cleanup...")
            X_clean = X_clean.fillna(X_clean.median())

        print(f"✅ Data validated: No infinite values, No missing values")

        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(
            X_clean, y, test_size=0.2, stratify=y, random_state=42
        )

        print(f"✅ Train/Test split: Train={X_train.shape}, Test={X_test.shape}")

        # Scale features
        scaler = StandardScaler()
        try:
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)
            print("✅ Feature scaling successful")
        except Exception as e:
            print(f"❌ Scaling failed: {e}")
            continue

        # Train models
        models = {
            'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
            'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
            'XGBoost': XGBClassifier(random_state=42, eval_metric='logloss')
        }

        feature_results = {}

        for model_name, model in models.items():
            print(f"\n🔄 Training {model_name}...")

            try:
                # Train model
                if model_name == 'Logistic Regression':
                    model.fit(X_train_scaled, y_train)
                    y_pred = model.predict(X_test_scaled)
                    y_pred_proba = model.predict_proba(X_test_scaled)
                else:
                    model.fit(X_train, y_train)
                    y_pred = model.predict(X_test)
                    y_pred_proba = model.predict_proba(X_test)

                # Calculate metrics
                accuracy = accuracy_score(y_test, y_pred)
                f1_weighted = f1_score(y_test, y_pred, average='weighted')

                print(f"✅ {model_name} Results:")
                print(f"   📊 Accuracy: {accuracy:.4f}")
                print(f"   📊 F1-Score (weighted): {f1_weighted:.4f}")

                # Store results
                feature_results[model_name] = {
                    'model': model,
                    'accuracy': accuracy,
                    'f1_score': f1_weighted,
                    'scaler': scaler if model_name == 'Logistic Regression' else None,
                }

            except Exception as e:
                print(f"❌ {model_name} training failed: {e}")
                continue

        results[feature_set_name] = feature_results

        # Print summary
        print(f"\n📈 SUMMARY FOR {feature_set_name.upper()}:")
        for model_name, result in feature_results.items():
            print(f"   {model_name:20}: Accuracy={result['accuracy']:.4f}, F1={result['f1_score']:.4f}")

    print(f"\n{'='*60}")
    print("🎉 SAFE TRAINING COMPLETE!")
    print(f"{'='*60}")

    return results




In [84]:
def run_safe_pipeline():
    """Execute safe Phase 2 pipeline with data validation"""

    try:
        print("🚀 STARTING SAFE PHASE 2 PIPELINE")
        print("="*60)

        # Step 1: Update VLE features with safe calculations
        print("\n1️⃣ RECREATING VLE FEATURES SAFELY...")
        vle_features_safe = create_vle_features_safe(oulad_data_clean['student_vle'])

        # Step 2: Recreate combined dataset with safe features
        print("\n2️⃣ RECREATING COMBINED DATASET...")
        combined_safe = oulad_data_clean['student_info'].copy()

        # Add registration data
        registration = oulad_data_clean['student_registration'][
            ['code_module', 'code_presentation', 'id_student', 'completed_course', 'withdrawal_status']
        ]
        combined_safe = combined_safe.merge(registration, on=['code_module', 'code_presentation', 'id_student'], how='inner')

        # Add safe VLE features
        combined_safe = combined_safe.merge(vle_features_safe, on='id_student', how='left')

        # Add assessment features
        assessment_features = create_assessment_features(oulad_data_clean['student_assessment'])
        combined_safe = combined_safe.merge(assessment_features, on='id_student', how='left')

        print(f"✅ Safe combined dataset: {combined_safe.shape}")

        # Step 3: Prepare modeling data
        print("\n3️⃣ PREPARING SAFE MODELING DATA...")
        processed_data_safe, all_features, behavior_only, target_encoder, le_dict = prepare_modeling_data(combined_safe)

        print(f"✅ Safe processed data: {processed_data_safe.shape}")

        # Step 4: Train models safely
        print("\n4️⃣ TRAINING MODELS SAFELY...")
        feature_sets = {
            'all_features': all_features,
            'behavior_only': behavior_only
        }

        model_results = train_baseline_models_safe(processed_data_safe, feature_sets)

        return model_results, processed_data_safe

    except Exception as e:
        print(f"❌ Safe pipeline error: {e}")
        import traceback
        traceback.print_exc()
        return None, None

# RUN THE SAFE PIPELINE
print("EXECUTING SAFE PIPELINE...")
model_results, processed_data_safe = run_safe_pipeline()


EXECUTING SAFE PIPELINE...
🚀 STARTING SAFE PHASE 2 PIPELINE

1️⃣ RECREATING VLE FEATURES SAFELY...
🔄 Creating VLE features with safe calculations...
✅ VLE features created for 26074 students

2️⃣ RECREATING COMBINED DATASET...
✅ Safe combined dataset: (32593, 29)

3️⃣ PREPARING SAFE MODELING DATA...
✅ Safe processed data: (32593, 30)

4️⃣ TRAINING MODELS SAFELY...
🚀 STARTING SAFE BASELINE MODEL TRAINING
Dataset shape: (32593, 30)
Target distribution:
target
2    12361
3    10156
1     7052
0     3024
Name: count, dtype: int64

🎯 TRAINING ON ALL_FEATURES FEATURE SET
✅ Initial data: X=(32593, 15), y=(32593,)
🧹 CLEANING INFINITE AND EXTREME VALUES
   ✅ Filled total_clicks missing values with median: 903.00
   ✅ Filled avg_clicks_per_session missing values with median: 3.13
   ✅ Filled active_days missing values with median: 55.00
   ✅ Filled daily_engagement_rate missing values with median: 0.30
   ✅ Filled avg_assessment_score missing values with median: 75.71
   ✅ Filled total_assessmen